### Data preprocessing of Find it Fix It requests to the City of Seattle (from a FOI request)

In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("poster")
sns.set_style("dark")
sns.set_style("ticks")
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.text import Text

import gensim
from gensim import corpora


In [2]:
#path for FIFI FOI request
path = 'C:/Users/Schindler/Documents/ProgrammingFun/FIFI/FIFI_data_through_5-12-19.xlsx'

In [3]:
#original file has multiple sheets (for different FIFI types)
whole_file = pd.ExcelFile(path)

print(whole_file.sheet_names)

combined_data = pd.DataFrame()
for sheet_name in whole_file.sheet_names:
    sheet_data = pd.DataFrame(data = pd.read_excel(whole_file, sheetname=sheet_name))
    print(sheet_name)
    print(sheet_data.columns.values)
    sheet_data['FIFI_category'] = [sheet_name]*sheet_data.shape[0]
    combined_data = pd.concat([combined_data, sheet_data], sort=False)

combined_data.reset_index(drop=True, inplace=True)
print(combined_data.shape)
combined_data.head(1)

['Abandoned Vehicle', 'Clogged Storm Drain', 'Community Walk', 'Dead Animal', 'Graffiti Report', 'Litter-Illegal Dumping-Needles', 'Overgrown Vegetation', 'Parking Enforcement', 'Pothole', 'Sign and Signal Maintenance', 'Streetlight Report', 'Other Inquiry']
Abandoned Vehicle
['Service Request Number' 'Created Date' 'Location' 'Location Details'
 'Description' 'License Number' 'State' 'Vehicle Make' 'Vehicle Color'
 'Length of Time Parked' 'Photo']
Clogged Storm Drain
['Service Request Number' 'Created Date' 'Location' 'Location Details'
 'Description' 'Location Details.1' 'Causing property damage?' 'Photo']
Community Walk
['Service Request Number' 'Created Date' 'Location' 'Location Details'
 'Description' 'Current Issue' 'Location Details.1' 'Photo']
Dead Animal
['Service Request Number' 'Created Date' 'Location' 'Location Details'
 'Description' 'Type of animal' 'On private property?' 'Photo']
Graffiti Report
['Service Request Number' 'Created Date' 'Location' 'Location Details'
 'D

,Service Request Number,Created Date,Location,Location Details,Description,License Number,State,Vehicle Make,Vehicle Color,Length of Time Parked,Photo,FIFI_category,Location Details.1,Causing property damage?,Current Issue,Type of animal,On private property?,What is the Graffiti on?,Location and Description,"Obscene, racial, or hateful?",Cross street or intersection?,Where?,Type of item?,Where is the vegetation growing,Vegetation Safety Issue,Parking Violation Concerning,License Plate Number,Location details,7 digit number on pole,Issue
0,13-00036779,2013-05-09 20:32:33,"5208 BEACON AVE S, SEATTLE, WA","5200 Beacon Ave S Seattle, WA 98108; XY: 12775...",Ford light blue,AHZ9829,NaN,Ford,Blue,Over 3 Days,NaN,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#clean and preprocess data
print('Combined data shape: ', combined_data.shape, '\n')

#create date column and bins
combined_data['date'] = combined_data['Created Date'].dt.date
#create study date bins
#combined_data['date_bin'] = pd.cut(combined_data['date'], 36)
#create year and month columns 
combined_data['year'] = combined_data['Created Date'].dt.year
combined_data['month'] = combined_data['Created Date'].dt.month

#select data that already includes a zip code (starting with 2015)
combined_data_2015 = combined_data[combined_data['Created Date'].dt.year > 2015]
print('Combined data 2015-2019 shape: ', combined_data_2015.shape, '\n')

#create new column with zip code
combined_data_2015['zip'] = combined_data_2015.apply(lambda row: row['Location'].split()[-1], axis=1)  

#select zips that have great than 999 entries
zip_counts = combined_data_2015['zip'].value_counts()
zip_counts_1000 = zip_counts[zip_counts > 999].index.values
combined_data_final = combined_data_2015[combined_data_2015['zip'].isin(zip_counts_1000)]
print('Combined data final zips: ', combined_data_final['zip'].value_counts(), '\n')

#select FIFI categories that have great than 999 entries
cat_counts = combined_data_final['FIFI_category'].value_counts()
cat_counts_1000 = cat_counts[cat_counts > 999].index.values
combined_data_final = combined_data_final[combined_data_final['FIFI_category'].isin(cat_counts_1000)]
print('Combined data final FIFI categories: ', combined_data_final['FIFI_category'].value_counts(), '\n')  

#update FIFI_cat names
combined_data_final.replace({'Litter-Illegal Dumping-Needles': 'Needles/Dumping', 
                             'Other Inquiry': 'Other',
                             'Abandoned Vehicle': 'Abnd_Vehicle',
                             'Parking Enforcement': 'Parking',
                             'Graffiti Report': 'Grafitti',
                             'Sign and Signal Maintenance': 'Sign/Signal',
                             'Streetlight Report': 'Streetlight'}, inplace=True)

#viz FIFI requests by category
combined_data_final['FIFI_category'].value_counts().plot(kind='bar')
plt.title('FIFI category value counts')
plt.show()
print('\n')

#viz FIFI requests by year
combined_data_final['year'].value_counts().plot(kind='bar')
plt.title('FIFI requests by year')
plt.show()

combined_data_final.head(1)

Combined data shape:  (248885, 33) 



TypeError: unsupported operand type(s) for +: 'datetime.date' and 'float'

In [10]:
#create dictionary of zips and corresponding neighborhoods, add neighborhood as new column

zip_neighborhood_dic = {'98028': 'Kenmore',  
                        '98031': 'Kent',
                        '98057': 'Renton',
                        '98101': 'Dt/FirstH', 
                        '98102': 'Cap_Hill', 
                        '98103': 'GnWd/Frmt/GnLk', 
                        '98104': 'PioneerS/FirstH',
                        '98105': 'Udist/LarHur', 
                        '98106': 'Delridge',
                        '98107': 'Ballard', 
                        '98108': 'South_Park',
                        '98109': 'Westlake',
                        '98112': 'Madison_Park',
                        '98115': 'Wedgewood',
                        '98116': 'WestS_Alki', 
                        '98117': 'Ballard',
                        '98118': 'ColCity/SewardP', 
                        '98119': 'Queen_Anne',
                        '98121': 'Belltown',
                        '98122': 'Leschi',
                        '98125': 'Northgate', 
                        '98126': 'Delridge',
                        '98133': 'Bitterlake',
                        '98134': 'Harbor_Island',
                        '98136': 'Fauntleroy',
                        '98144': 'MtBaker', 
                        '98146': 'Arbor_Heights',
                        '98148': 'Sunnydale',
                        '98155': 'North_City',
                        '98164': 'DT_library',
                        '98166': 'Gregory_Heights',
                        '98168': 'Burien',
                        '98177': 'LoyalH/Highlands',
                        '98178': 'Lakeridge',
                        '98188': 'SeaTac',
                        '98195': 'UW_campus',
                        '98199': 'Magnolia'}

zip_list = list(zip_neighborhood_dic.keys())
zip_filtered = combined_data_final[combined_data_final['zip'].isin(zip_list)]
print(zip_filtered.shape)

zip_filtered['neighborhood'] = zip_filtered.apply(lambda row: zip_neighborhood_dic[row['zip']], axis=1)

zip_filtered.head(1)

(202702, 34)


,Service Request Number,Created Date,Location,Location Details,Description,License Number,State,Vehicle Make,Vehicle Color,Length of Time Parked,Photo,FIFI_category,Location Details.1,Causing property damage?,Current Issue,Type of animal,On private property?,What is the Graffiti on?,Location and Description,"Obscene, racial, or hateful?",Cross street or intersection?,Where?,Type of item?,Where is the vegetation growing,Vegetation Safety Issue,Parking Violation Concerning,License Plate Number,Location details,7 digit number on pole,Issue,year,month,date,zip,neighborhood
5994,16-00000050,2016-01-01 10:17:27,"5900 34TH CT S, SEATTLE, WA 98118","5900 34TH CT S; XY: 1280854.342623405, 203954....","Gold car, appears to have a window that won't ...",065-XVB,WA,Mercedes-Benz,Other,Over 1 month,http://servicerequest.seattle.gov/media/seattl...,Abnd_Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,1,2016-01-01,98118,ColCity/SewardP


In [11]:
zip_filtered.to_pickle('cleaned_data.pkl')